In [0]:
#Gold table creation

silver_df = spark.table("workspace.silver.wimbledon_match_first_point_winner_2023")

display(silver_df.limit(10))
silver_df.count()

In [0]:
#filter rally length
from pyspark.sql.functions import col

rally_df = silver_df.filter(col("RallyCount") > 3)

display(rally_df.limit(10))
rally_df.count()

In [0]:
#Compare point winner to match winner
from pyspark.sql.functions import when, count, lit, col, round

analysis_df = rally_df.withColumn("first_point_wins_match",
                                  when(col("first_point_winner") == col("match_winner"), True)
                                  .otherwise(False)
                                  )

#display(analysis_df.limit(10))

total = analysis_df.count()

win_rate_df = (
    analysis_df
    .groupBy("first_point_wins_match")
    .agg(
        (count("match_id") / lit(total)).alias("win_percentage")
    )
)

win_rate_df = win_rate_df.withColumn(
    "win_percentage",
    round(col("win_percentage") * 100, 2)
)

display(win_rate_df)

In [0]:
#Create schema and table
spark.sql("CREATE SCHEMA IF NOT EXISTS workspace.gold")

(
    win_rate_df.write
    .format("delta")
    .mode("overwrite")
    .saveAsTable("workspace.gold.wimbledon_match_first_point_winner_rate_2023")
)
